In [3]:
import csv
from django.db import connection
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from batid.models import Building
# from django.contrib.gis.geos import GEOSGeometry
# from django.contrib.gis.db.models.functions import Distance
import pandas as pd

df = pd.DataFrame()
with open('test_arretes_insa_2023_rnb.geocoded.csv', newline='') as csvfile:
    df = pd.read_csv(csvfile)
    # filter the df with the lowest result_score
    df_poor_geocoding = df.sort_values(by=['result_score'])
    print(df_poor_geocoding[['gid', 'result_score', 'num_voie',	'ind_rep',	'nom_voie',	'code_postal',	'commune', 'result_label']].head(20))

     gid  result_score  num_voie ind_rep  \
48    42      0.572445         4     NaN   
49    43      0.572445         4     NaN   
44    40      0.693765      5094     NaN   
101   98      0.694023         3     NaN   
78    76      0.694023         3     NaN   
74    69      0.768660         7     NaN   
68    62      0.768801       NaN     NaN   
45    41      0.819088        26     NaN   
82    78      0.821048         7     NaN   
18    28      0.821048         7     NaN   
60    55      0.821048         7     NaN   
29    31      0.838524  76-78-80     NaN   
85    82      0.838524  76-78-80     NaN   
69    63      0.838524  76-78-80     NaN   
51    47      0.867625         1       A   
3    109      0.869314        24       B   
58    53      0.871664       129     NaN   
50    45      0.875895        74     NaN   
90    87      0.894869        89     NaN   
13    44      0.899561         1     NaN   

                                      nom_voie  code_postal          commun

Il semble sur cet échantillon que tous les résultats dont le score BAN est inférieur à 75% sont faux. Nous allons donc les exclure.

In [22]:
df_valid_geocoding = df[(df['result_score'] > 0.75) & (df['result_type'] == 'housenumber')]
print(len(df_valid_geocoding))
print('adresses sur')
print(len(df))
print('initiales sont gardées')

104
adresses sur
117
initiales sont gardées


In [23]:
def get_buildings_from_ban_id(ban_id):
    buildings = Building.objects.filter(addresses__id=ban_id)
    return [b.rnb_id for b in buildings]

# call the function for each row of the dataframe
df_valid_geocoding['rnb_id_from_ban_id'] = df_valid_geocoding['result_id'].apply(get_buildings_from_ban_id)

print(df_valid_geocoding[['gid', 'rnb_id_from_ban_id']].head(20))


    gid                                 rnb_id_from_ban_id
0    46                       [DGSYNWEXS5AZ, QE86Y194MZHX]
1    65                       [FGHSF5PTF4ZY, TCSN8X19NNSA]
2    80                                     [TSTMGJVZXZ56]
3   109                       [1D5156HMPQFH, ZK61ZY35424K]
4   110                       [QHBXCBBP32GQ, ZEC4KC5N57J3]
5    18                                     [8835STW2T6X7]
6    19                                     [SSJW6GQ5GHD2]
7    20                                     [QWPV5DJY4J8C]
8   108                                     [ET7T8PEX5J7H]
9    21                       [65PRGM9NQTJR, XJHY1B48X6A9]
10   22  [49QT7Z6FDGW6, 7CY7E5T65N7M, GBF486JWVSJQ, HVV...
12   24  [1XDPJ8NAK7K6, 1ZF7DP9NSCK6, 2Z28ZCT4PQQS, 4MK...
13   44                                     [19PH91AG6M7K]
14    7                                     [B5BVR2TFKYJW]
15   25                                     [1PEABG4EK95E]
16   26                       [H8BM73235NV3, MKCZ7V7PZNB

/tmp/ipykernel_1215/4065385847.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_geocoding['rnb_id_from_ban_id'] = df_valid_geocoding['result_id'].apply(get_buildings_from_ban_id)


Combien trouvons nous de batiment(s) pour chaque adresse localisée ?

In [24]:
df_valid_geocoding['rnb_id_from_ban_id'].apply(lambda x: len(x)).value_counts()

# select the row where we have 33 buildings
#df_valid_geocoding[df_valid_geocoding['rnb_id_from_ban_id'].apply(lambda x: len(x)) == 33]

1     59
2     24
4      6
3      4
5      2
8      2
6      1
29     1
0      1
33     1
7      1
11     1
10     1
Name: rnb_id_from_ban_id, dtype: int64

In [28]:
# merge the results with the original dataframe
df = df.merge(df_valid_geocoding[['gid', 'rnb_id_from_ban_id']], on='gid', how='left')

# save the results
df.to_csv('test_arretes_insa_2023_rnb.rapproche.csv', index=False)

In [27]:
df_requests = df_valid_geocoding['result_id'].apply(lambda x: f"select * from batid_building bb left join batid_building_addresses bba on bba.building_id  = bb.id where bba.address_id = '{x}';")

# write the result in a csv file
df_requests.to_csv('test_arretes_insa_2023_rnb.geocoded.valid.requests.csv', index=False)